In [132]:
import argparse
import os
import re
import pandas as pd
import numpy as np

In [178]:
def data_parse(case, port,):
    
    current_dir = os.getcwd()
    home_dir = os.path.abspath(os.path.join(current_dir, '..'))
    data_dir = os.path.join(home_dir, 'Data')
    case_dir = os.path.join(data_dir, case)
    excel_dir = os.path.join(data_dir, '%s.xlsx' % case)
    result_dir = os.path.join(data_dir, '%s' % case)
    
    para_df = pd.read_excel(excel_dir, sheet_name='Mixed_N_Line_Stripline', skiprows=23)
    
    # Set new df
    new_para_df = para_df.T.loc[para_df.T.index[1:]].copy()
    new_para_df.columns = para_df.T.loc['batch list'].to_numpy()
    new_para_df = new_para_df.set_index('save dir file name')
    new_para_df.index.name = None
    
    
    # Get the absolute path of the upper directory

    # rlgc_df_list = []
    snp_df_list = []
    keys = []
    snp_headers = []
    
    for i in range(port):
        for j in range(port):
            for s in 'SR', 'SI':
                snp_headers.append('%s(%d,%d)' % (s, i+1, j+1))

    for dirs in os.listdir(result_dir):
                
        # skip with
        if os.path.isdir(os.path.join(result_dir, dirs, 'RLGC')):
            
            # read rlgc
            # rlgc = 'TransmissionLine.rlgc'
            
            # read snp
            snp = 'TransmissionLine.s%dp' % port
            
            if os.path.exists(os.path.join(result_dir, dirs, 'RLGC', snp)):
                keys.append(dirs)
                # rlgc_df_list.append(pd.read_csv(os.path.join(data_path, dirs, rlgc), skiprows=2, delim_whitespace=True))

                snp_df = pd.read_csv(os.path.join(result_dir, dirs, 'RLGC', snp), skiprows=port+3, delim_whitespace=True, header=None).loc[:, 1:]
                snp_df.columns = snp_headers
                
                for p in new_para_df.columns:
                    if p == 'W':
                        snp_df.loc[:, p] = float(new_para_df.loc[dirs, p].split(',')[0])
                    else:
                        snp_df.loc[:, p] = float(new_para_df.loc[dirs, p])
                
                snp_df_list.append(snp_df)
                
    # df = pd.concat([pd.concat(rlgc_df_list, keys=keys), pd.concat(snp_df_list, keys=keys)], axis=1)
    df = pd.concat(snp_df_list, keys=keys)

    
    return df

In [179]:
new_df = data_parse('Differential_Stripline_test', port=4)